In [2]:
dirct = r'D:\python_code\M5 Competition\m5-forecasting-Dataset\\'
path2 = r'D:\python_code\M5 Competition\A5_solution\A5_output2\\'

In [ ]:
import numpy as np
import pandas as pd 
import pdcast as pdc
import pickle
import os
for dirname, _, filenames in os.walk(dirct):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.options.display.max_columns = 50

import random
from  datetime import datetime, timedelta

import lightgbm as lgb
import joblib

D:\python_code\M5 Competition\m5-forecasting-Dataset\\calendar.csv
D:\python_code\M5 Competition\m5-forecasting-Dataset\\data.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelCA_1.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelCA_2.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelCA_3.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelCA_4.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelTX_1.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelTX_2.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelTX_3.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelWI_1.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelWI_2.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\modelWI_3.pkl
D:\python_code\M5 Competition\m5-forecasting-Dataset\\sales_train_evaluation.csv
D:\python_code\M5 Competition\m5-forecasting-Dataset\\sales_train_evaluation2.csv
D:\python_code\M5 Competiti

In [7]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32", 
                "rm_diff_price_4":"float32", "rm_diff_price_12":"float32","rm_diff_price_50":"float32" }
PROC_PRICES_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16", 
                        "rm_diff_price_4":"float32", "rm_diff_price_12":"float32","rm_diff_price_50":"float32" }

In [4]:
h = 28 
max_lags = 70
tr_last = 1913
fday = datetime(2016,4,25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [ ]:
## downcast sales data
sales_df = pd.read_csv(dirct + "sales_train_validation.csv")
sales_df = pdc.downcast(sales_df)



In [8]:
prices = pd.read_csv(dirct + "sell_prices.csv", dtype = PRICE_DTYPES)
prices = pdc.downcast(prices)
proc_price = pd.read_csv('D:\python_code\M5 Competition\A5_solution\A5_output\\prices_processed.csv', dtype = PROC_PRICES_DTYPES).drop('sell_price', axis=1)
proc_price = pdc.downcast(proc_price)
prices = prices.merge(proc_price, on=['store_id','item_id','wm_yr_wk'], how='left')
del(proc_price)

for col, col_dtype in PRICE_DTYPES.items():
    if col_dtype == "category":
        prices[col] = prices[col].cat.codes.astype("int16")
        prices[col] -= prices[col].min()
        
cal = pd.read_csv(dirct + "calendar.csv", dtype = CAL_DTYPES)
proc_cal = pd.read_csv('D:\python_code\M5 Competition\A5_solution\A5_output\\processed_calendar.csv').drop(
['wm_yr_wk','wday','month','year','snap_CA','snap_TX','snap_WI'], axis=1).rename(columns={'day':'d'})
cols_events_days = ['d_{}'.format(c) for c in list(np.arange(1910,1990))]
ev1 = cal[cal['d'].isin(cols_events_days)]['event_name_1'].unique().tolist()
ev2 = cal[cal['d'].isin(cols_events_days)]['event_name_2'].unique().tolist()
evs = list(set(ev1+ev2))
for c in list(set(proc_cal.columns.tolist()) - set(['d'])):
    proc_cal[c] = proc_cal[c].astype(int)
cal = cal.merge(proc_cal, on='d', how='left')
del(proc_cal)
cal["date"] = pd.to_datetime(cal["date"])
for col, col_dtype in CAL_DTYPES.items():
    if col_dtype == "category":
        cal[col] = cal[col].cat.codes.astype("int16")
        cal[col] -= cal[col].min()
events_to_mantain = ['event_name_1_{}'.format(c) for c in evs]+['event_name_2_{}'.format(c) for c in evs]
events_to_mantain_ = [c for c in cal.columns if c in events_to_mantain]
cal = cal[['date','wm_yr_wk','weekday','wday','month','year','d','event_name_1','event_type_1','event_name_2',
            'event_type_2','snap_CA','snap_TX','snap_WI','group_day']+events_to_mantain_]



<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:14: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:14: SyntaxWarning: invalid escape sequence '\p'
C:\Users\kambl\AppData\Local\Temp\ipykernel_9120\4269749393.py:3: SyntaxWarning: invalid escape sequence '\p'
C:\Users\kambl\AppData\Local\Temp\ipykernel_9120\4269749393.py:14: SyntaxWarning: invalid escape sequence '\p'


In [ ]:
def create_dt(is_train = True, nrows = None, first_day = 1200, dept='HOBBIES_1'):
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    # dt = pd.read_csv(dirct + "sales_train_validation_down.csv", 
    #                  nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    dt = sales_df[catcols + numcols].astype(dtype)
    dt = dt[dt['dept_id']==dept]
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:14: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:14: SyntaxWarning: invalid escape sequence '\p'
C:\Users\kambl\AppData\Local\Temp\ipykernel_19648\2132835270.py:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\kambl\AppData\Local\Temp\ipykernel_19648\2132835270.py:14: SyntaxWarning: invalid escape sequence '\p'


In [15]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "week",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        elif date_feat_name in ['year', 'week', 'mday']:
            dt[date_feat_name] = getattr(dt["date"].dt.isocalendar(), date_feat_func).astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [16]:
FIRST_DAY = 350

In [ ]:
%%time
for dept in ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']:

    df = create_dt(is_train=True, first_day= FIRST_DAY, dept=dept)
    print(df.shape)
    create_fea(df)
    print(df.shape)

    for c in [c for c in df.columns.tolist() if 'rm_diff_price_' in c]:
        df[c].fillna(0, inplace=True)
    
    df.shape

    cat_feats = (['item_id','store_id', 'cat_id', 'state_id'] 
                 + ["event_type_1", "event_type_2"]
                 + ['wday', 'month', 'snap_CA', 'snap_TX', 'snap_WI'])
    
    useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", "dept_id", "sell_price",'event_name_1', 'event_name_2']
    train_cols = df.columns[~df.columns.isin(useless_cols)]

    days_val = random.choices(df['d'].unique().tolist(), k=500)
    X_train = df[df['d'].isin(days_val)==False][train_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    y_train = df[df['d'].isin(days_val)==False]["sales"].replace([np.inf, -np.inf], np.nan).fillna(0)
    X_val = df[df['d'].isin(days_val)==True][train_cols]
    y_val = df[df['d'].isin(days_val)==True]["sales"]

    train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats)
    valid_data = lgb.Dataset(X_val, label = y_val, categorical_feature=cat_feats)

    params = {
            "objective" : "poisson",
            "metric" :"poisson",
            "learning_rate" : 0.09,
            "sub_feature" : 0.9,
            "sub_row" : 0.75,
            "bagging_freq" : 1,
            "lambda_l2" : 0.1,
            'verbosity': 1,
            'num_iterations' : 2000,
            'num_leaves': 32,
            "min_data_in_leaf": 50,
    }

    # m_lgb = lgb.train(params, train_data, valid_sets = [train_data, valid_data], 
    #                   verbose_eval=20, early_stopping_rounds=30) 
    m_lgb = lgb.train(params, train_data, valid_sets = [train_data, valid_data])

    ######### save the model
    # m_lgb.booster_.save_model(path2 + 'lgbm_model_HOBBIES_1.txt')
    filename = path2 + 'lgbm_model_' + dept + '.pkl'
    # pickle.dump(m_lgb, open(file, 'wb'))
    joblib.dump(m_lgb, filename)

In [ ]:
%%time

sub_p_total = pd.DataFrame()
for dept in ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']:
    useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", "dept_id", "sell_price",'event_name_1', 'event_name_2']
    train_cols = df.columns[~df.columns.isin(useless_cols)]

    for fn in os.listdir(path2):
        if 'model' in fn and dept in fn and fn.endswith('.pkl'):                # added
            model_fn = fn
            m_lgb = joblib.load(path2 + fn)
    print(f'prediction started for {model_fn}')
    te = create_dt(False, dept=dept)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
   
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
    
        te.loc[te.date == day, "sales"] = m_lgb.predict(tst)
  
        del(tst)

    del(m_lgb)
    sub_p = pd.pivot_table(te, index='id', values='sales', columns='d').iloc[:,-28:].reset_index()

    del(te)
    sub_p_total = pd.concat([sub_p_total, sub_p])
    del(sub_p)
    print(f'prediction done for {model_fn}')

prediction started for lgbm_model_HOBBIES_1.pkl
prediction done for lgbm_model_HOBBIES_1.pkl
prediction started for lgbm_model_HOBBIES_2.pkl
prediction done for lgbm_model_HOBBIES_2.pkl
prediction started for lgbm_model_HOUSEHOLD_1.pkl
prediction done for lgbm_model_HOUSEHOLD_1.pkl
prediction started for lgbm_model_HOUSEHOLD_2.pkl
prediction done for lgbm_model_HOUSEHOLD_2.pkl
prediction started for lgbm_model_FOODS_1.pkl
prediction done for lgbm_model_FOODS_1.pkl
prediction started for lgbm_model_FOODS_2.pkl
prediction done for lgbm_model_FOODS_2.pkl
prediction started for lgbm_model_FOODS_3.pkl
prediction done for lgbm_model_FOODS_3.pkl
CPU times: total: 9min 38s
Wall time: 9min 40s


In [68]:
xdata[1:200]

,item_id,store_id,cat_id,state_id,wday,month,year,event_type_1,event_type_2,snap_CA,snap_TX,snap_WI,group_day,event_name_1_Cinco De Mayo,event_name_1_Father's day,event_name_1_MemorialDay,event_name_1_Mother's day,event_name_1_NBAFinalsEnd,event_name_1_NBAFinalsStart,event_name_1_OrthodoxEaster,event_name_1_Pesach End,event_name_1_Ramadan starts,event_name_2_Cinco De Mayo,event_name_2_Father's day,event_name_2_OrthodoxEaster,rm_diff_price_1,rm_diff_price_2,rm_diff_price_4,rm_diff_price_8,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
1,3,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
2,4,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
3,7,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
4,8,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
5,9,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,348,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
196,351,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
197,354,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5
198,355,0,0,0,7,1,2012,0,0,0.0,1.0,0.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,5


In [42]:
sub_p_total.shape

(30490, 29)

In [43]:
sub_p_total.tail()

d,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
8225,FOODS_3_827_TX_2_validation,0.116703,0.115136,2.532196,1.246543,1.314479,1.620670,1.569215,1.244027,1.248251,0.829346,0.738927,0.892698,1.274189,1.199256,1.196751,1.080407,1.037131,1.039852,1.060964,1.222252,1.269638,0.931099,0.886670,0.878874,0.902704,0.989324,1.238691,1.296787
8226,FOODS_3_827_TX_3_validation,1.124752,1.144748,0.990996,1.097953,1.016126,1.132709,1.413633,0.934777,0.930433,0.868771,0.821721,1.028875,1.200069,0.946486,1.036114,0.893256,0.914942,0.937564,0.942051,0.999114,1.017432,0.792745,0.738679,0.717460,0.720472,0.753455,0.853041,0.976115
8227,FOODS_3_827_WI_1_validation,3.197606,3.833251,3.605053,3.288888,3.703484,4.594605,4.180150,2.975678,2.657641,2.736156,2.685023,3.802427,4.422973,3.704359,3.213874,2.756903,2.789101,2.582462,3.024180,3.725732,3.600239,2.481687,2.462077,2.448491,2.516117,2.987611,3.472968,3.440767
8228,FOODS_3_827_WI_2_validation,2.456243,2.644096,2.622829,2.615265,2.706612,3.089757,3.441386,3.269370,3.138004,2.462212,2.822475,3.348898,3.101315,2.962391,3.082713,2.419391,2.947842,2.933584,2.580724,3.318765,3.005474,2.111658,2.002661,2.048486,2.110220,2.126098,2.278061,2.411067
8229,FOODS_3_827_WI_3_validation,1.390866,1.376567,1.190643,1.306089,1.407538,1.811310,1.336769,1.350513,1.363357,1.317863,1.439487,1.863064,1.966515,1.464617,1.507369,1.137497,1.273454,1.269857,1.367890,1.851430,1.721016,1.255225,1.080520,1.008811,0.972197,1.076091,1.360186,1.377460


In [44]:
sub = pd.read_csv(dirct + 'sample_submission.csv', usecols=['id'])

In [45]:
sub = sub.merge(sub_p_total, on='id', how='left')

In [46]:
sub = sub.dropna()

In [47]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.columns = ['id'] + ['F' + str(c) for c in np.arange(1,29,1)]
sub.to_csv(path2 + "partial_submission_18nov.csv",index=False)